In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import torch
import argparse
from models import EncoderCNN, SentenceLSTM, WordLSTM
from collections import Counter

class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)


from dataloader import get_loader
from score import evalscores
from torchvision import transforms
from torch import nn
import numpy as np
from nltk.translate.bleu_score import corpus_bleu

[1000/6473] Tokenized the captions.
[2000/6473] Tokenized the captions.
[3000/6473] Tokenized the captions.
[4000/6473] Tokenized the captions.
[5000/6473] Tokenized the captions.
[6000/6473] Tokenized the captions.


In [2]:
from cococaption.pycocotools.coco import COCO
from cococaption.pycocoevalcap.eval import COCOEvalCap
import json

In [3]:
import pandas as pd

In [4]:
from collections import Counter

class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)


In [5]:
!nvidia-smi

Thu Jun  4 21:28:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  Off  | 00000000:07:00.0 Off |                    0 |
| N/A   47C    P0   104W / 300W |  13718MiB / 16155MiB |     42%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  Off  | 00000000:08:00.0 Off |                    0 |
| N/A   48C    P0   118W / 300W |  16010MiB / 16158MiB |     32%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [6]:
from train import script

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument('--img_size', type = int, default = 224, help = 'size to which image is to be resized')
parser.add_argument('--crop_size', type = int, default = 224, help = 'size to which the image is to be cropped')
parser.add_argument('--device_number', type = str, default = "0", help = 'which GPU to run experiment on')


parser.add_argument('--int_stop_dim', type = int, default = 64, help = 'intermediate state dimension of stop vector network')
parser.add_argument('--sent_hidden_dim', type = int, default = 512, help = 'hidden state dimension of sentence LSTM')
parser.add_argument('--sent_input_dim', type = int, default = 1024, help = 'dimension of input to sentence LSTM')
parser.add_argument('--word_hidden_dim', type = int, default = 512, help = 'hidden state dimension of word LSTM')
parser.add_argument('--word_input_dim', type = int, default = 512, help = 'dimension of input to word LSTM')
parser.add_argument('--att_dim', type = int, default = 64, help = 'dimension of intermediate state in co-attention network')
parser.add_argument('--num_layers', type = int, default = 1, help = 'number of layers in word LSTM')


parser.add_argument('--lambda_sent', type = int, default = 1, help = 'weight for cross-entropy loss of stop vectors from sentence LSTM')    
parser.add_argument('--lambda_word', type = int, default = 1, help = 'weight for cross-entropy loss of words predicted from word LSTM with target words')


parser.add_argument('--batch_size', type = int, default = 8, help = 'size of the batch')
parser.add_argument('--shuffle', type = bool, default = True, help = 'shuffle the instances in dataset')
parser.add_argument('--num_workers', type = int, default = 0, help = 'number of workers for the dataloader')
parser.add_argument('--num_epochs', type = int, default = 1, help = 'number of epochs to train the model')
parser.add_argument('--learning_rate_cnn', type = int, default = 1e-5, help = 'learning rate for CNN Encoder')
parser.add_argument('--learning_rate_lstm', type = int, default = 5e-3, help = 'learning rate for LSTM Decoder')


parser.add_argument('--log_step', type=int , default=10, help='step size for prining log info')
parser.add_argument('--save_step', type=int , default=1000, help='step size for saving trained models')

_StoreAction(option_strings=['--save_step'], dest='save_step', nargs=None, const=None, default=1000, type=<class 'int'>, choices=None, help='step size for saving trained models', metavar=None)

In [8]:
args = parser.parse_args('')

In [9]:
# os.environ["CUDA_VISIBLE_DEVICES"]= args.device_number
args.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(args.device)

cuda


In [10]:
args

Namespace(att_dim=64, batch_size=8, crop_size=224, device=device(type='cuda'), device_number='0', img_size=224, int_stop_dim=64, lambda_sent=1, lambda_word=1, learning_rate_cnn=1e-05, learning_rate_lstm=0.005, log_step=10, num_epochs=1, num_layers=1, num_workers=0, save_step=1000, sent_hidden_dim=512, sent_input_dim=1024, shuffle=True, word_hidden_dim=512, word_input_dim=512)

In [11]:
args, val_loader, encoderCNN, sentLSTM, wordLSTM, vocab = script(args)

vocab_size:  743
<start>
[['lungs are clear without focal consolidation effusion or pneumothorax.', 'normal heart size.', 'bony thorax and soft tissues grossly unremarkable.', 'calcified left hilar xxxx.']]
[]
Bleu_1: 0.000
Bleu_2: 0.000
Bleu_3: 0.000
Bleu_4: 0.000
ROUGE_L: 0.000
CIDEr: 0.000
Epoch [0/1], Step [0/810], Loss: 47.2834
Epoch [0/1], Step [10/810], Loss: 8.5567
Epoch [0/1], Step [20/810], Loss: 5.9406
Epoch [0/1], Step [30/810], Loss: 4.7145
Epoch [0/1], Step [40/810], Loss: 3.3391
Epoch [0/1], Step [50/810], Loss: 8.4368
Epoch [0/1], Step [60/810], Loss: 6.3655
Epoch [0/1], Step [70/810], Loss: 8.1142
Epoch [0/1], Step [80/810], Loss: 6.2800
Epoch [0/1], Step [90/810], Loss: 6.6269
Epoch [0/1], Step [100/810], Loss: 9.3041
Epoch [0/1], Step [110/810], Loss: 4.5227
Epoch [0/1], Step [120/810], Loss: 4.0043
Epoch [0/1], Step [130/810], Loss: 4.0183
Epoch [0/1], Step [140/810], Loss: 4.6532
Epoch [0/1], Step [150/810], Loss: 4.6831
Epoch [0/1], Step [160/810], Loss: 6.8671
Ep

### Теперь протестируем нашу модель на просто каком-то одном батче и посмотрим что выходит

In [12]:
encoderCNN

EncoderCNN(
  (cnn): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16):

In [13]:
encoderCNN.eval()
sentLSTM.eval()
wordLSTM.eval()

vocab_size = len(vocab)

In [14]:
for i, (images, captions, prob) in enumerate(val_loader):
    break

In [15]:
images.shape

torch.Size([8, 3, 224, 224])

In [16]:
captions.shape

torch.Size([8, 5, 18])

In [17]:
captions[:3, :, :10]

tensor([[[  1,  23,   4,   5,  25,  11,  30,   6, 415,   2],
         [  1,   7,   8,  10,  11,  12,   2,   0,   0,   0],
         [  1,  23,  13,   9, 719,   2,   0,   0,   0,   0],
         [  1,  32,  25,  15,  16,  18,  19,  17,  54,  55],
         [  1,  32,  25, 106,  28,  23,  76,  84,   2,   0]],

        [[  1,  23,  53,   6, 120,   9, 116,   2,   0,   0],
         [  1,  23,  38, 110,   9, 109,   2,   0,   0,   0],
         [  1,  32,  25,  42,  44,  45,  29,  23,  60, 167],
         [  1,  33,  34,  35,  17, 152,  29,  23,  37, 150],
         [  1,  32,  25,  15,  19,  17,  16,   2,   0,   0]],

        [[  1,  32,   9, 112, 134,  34,   6,  51, 398, 157],
         [  1,  23,  53,  25, 348, 158, 501,  28,  23, 121],
         [  1,  32,  25, 146,  28,  23, 145, 310, 158, 205],
         [  1,  32,  25,  15,  54,  96, 142,  72,  18,  19],
         [  1,  32, 358, 140,  84, 125,   2,   0,   0,   0]]])

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
images = images.to(args.device)
captions = captions.to(args.device)
prob = prob.to(args.device)

In [20]:
vis_enc_out = encoderCNN(images)

topics, ps = sentLSTM(vis_enc_out, torch.ones_like(captions), args.device)

# loss_sent = criterion_stop_val(ps.view(-1, 2), prob.view(-1))

# loss_word = torch.tensor([0.0]).to(args.device)

pred_words = torch.zeros((captions.shape[0], captions.shape[1], captions.shape[2])).to(args.device)

In [21]:
topics[:2, 0, :20]

tensor([[   0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000,    0.0000,    0.0000,    0.0000,    0.0000, 1178.4891,
            0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000, 1868.8875],
        [   0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000,    0.0000,    0.0000,    0.0000,    0.0000, 1056.9327,
            0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000, 1745.9353]], device='cuda:0', grad_fn=<SliceBackward>)

In [22]:
captions[:, 0, :2]

tensor([[ 1, 23],
        [ 1, 23],
        [ 1, 32],
        [ 1, 23],
        [ 1, 23],
        [ 1, 87],
        [ 1, 11],
        [ 1, 23]], device='cuda:0')

In [23]:
pred_words[:, :, 0] = 1
pred_words = pred_words.long()
pred_words.shape

torch.Size([8, 5, 18])

In [24]:
for i in range(captions.shape[1]):
    for j in range(2, captions.shape[2]):
        pred_words[:, i, :j] = wordLSTM(topics[:, i, :], pred_words[:, i, :j]).argmax(2)

In [25]:
pred_words[:, 0, :]

tensor([[ 1, 23, 13,  9, 14,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 23, 13,  9, 14,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 23, 13,  9, 14,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 23, 13,  9, 14,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 23, 13,  9, 14,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 23, 13,  9, 14,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 23, 13,  9, 14,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 23, 13,  9, 14,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
       device='cuda:0')

Как мы видим первые предложения совпали для каждого изображения в батче

Но они не лишены смысла:

In [26]:
for i in pred_words[0, 0, :]:
    print(vocab.idx2word[int(i)])

<start>
the
lungs
are
clear
<end>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>


In [27]:
for i in pred_words[0, 1, :]:
    print(vocab.idx2word[int(i)])

<start>
the
lungs
are
clear
<end>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
<pad>
